Udacity <a href="https://learn.udacity.com/nanodegrees/nd025/parts/cd1971/lessons/e3f2b25a-5e8a-404f-a1f8-18c3524ce867/concepts/e3f2b25a-5e8a-404f-a1f8-18c3524ce867-project-rubric">Rubric</a>


In [ ]:
# imports
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder  # for dealing with ordinal categories
from sklearn.preprocessing import (
    TargetEncoder,  # for dealing with high cardinal categories, for now use one-hot encoder
)
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import LinearSVR

%matplotlib inline


warnings.simplefilter(action="ignore", category=FutureWarning)

# Business Understanding

## Project Overview

This project will explore the Kaggle housing dataset.  

## Problem Statement

Explore how various feature selection methods influence predicted housing prices.

Questions to be answered in this project:
* What is the prediction score of a baseline simple linear regression model using all PCA components plus all original components? i.e. the case of no feature selection.
<br>
<br>
* Can some other simple linear regression models that contain feature select perform better such as:
    * Lasso
    * LassoCV
    * Linear Regression with select Kbest
<br>
<br>
* Can a more complex Random Forest Regressor model baseline w/out feature selection perform better than the more simple linear regression models? Baseline features will be PCA components plus all original components which is the same as the above simple linear regression.
<br>
<br>
* What happens when we apply various feature selection to the Random Forest Regressor? Does the model improve?
<br>
<br>
* Which model performed the best overall across all tested models?
<br>
<br>
* What insights can be made regarding how feature selection has impacted our prediction scores?
<br>
<br>
My hypothesis is that the best performing model overall will be the Random Forest Regressor with some kind of feature selection algorithm.  I think the Random Forest model will score better overall due to it being a more sophisticated model as compared to Linear Regression.  And I believe feature selection will further improve the performance as it will help prevent the Random Forest model from overfitting the training data.

## Metric

The metric that will be used to evaluate all models is the Root Mean Squared Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price.  As Kaggle notes taking the logs means that the errors in predicting expensive houses and cheap houses will affect the results equally.

Further comparisons will be made between the RMSE of the training data vs the testing data for the different complexities of the models.

# Data Understanding

# Data Preparation

# Data Modeling

# Result Evaluation

# Deployment